In [ ]:
!pip install roboflow ultralytics torchvision

from roboflow import Roboflow

print("🔗 Connecting to Roboflow...")
rf = Roboflow(api_key="blm6FIqi33eLS0ewVlKV")
project = rf.workspace("2642025").project("19-06")
version = project.version(7)

print("📥 Downloading dataset...")
dataset = version.download("yolov12")

In [ ]:
!gdown 1s5KprrhHWkbhjRWCb3OK48I-OriDLR_S

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms

# Load mô hình ResNet đã huấn luyện (ví dụ: ResNet18)
model = models.resnet50(weights=None)
num_classes = 4  # Sửa lại theo số class của bạn
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.load_state_dict(torch.load("resnet50_dog_head_emotion_4cls_50e_best_v1.pth", map_location=device))
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [ ]:
import os
import cv2
from pathlib import Path
dataset_path = Path("/content/19/06-7/test")
input_folder = Path(f"{dataset_path}/images/")  # Thư mục ảnh gốc
labels_folder = Path(f"{dataset_path}/labels/")
output_folder = dataset_path / "cropped_images"
output_folder.mkdir(exist_ok=True)

from PIL import Image

def crop_and_save(image_path, label_path):
    img = cv2.imread(str(image_path))
    h, w, _ = img.shape
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for idx, line in enumerate(lines):
            cls, x_center, y_center, bw, bh = map(float, line.strip().split())
            # Chuyển từ YOLO format sang pixel
            x1 = int((x_center - bw / 2) * w)
            y1 = int((y_center - bh / 2) * h)
            x2 = int((x_center + bw / 2) * w)
            y2 = int((y_center + bh / 2) * h)
            crop = img[y1:y2, x1:x2]
            crop_filename = output_folder / f"{image_path.stem}_{idx}.jpg"
            cv2.imwrite(str(crop_filename), crop)

for img_path in input_folder.glob("*.jpg"):
    label_path = labels_folder / (img_path.stem + ".txt")
    if label_path.exists():
        crop_and_save(img_path, label_path)


In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd

results = []  # Lưu kết quả để dùng ở Cell 2

for cropped_img in tqdm(sorted(output_folder.glob("*.jpg"))):
    img = Image.open(cropped_img).convert('RGB')
    input_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)
        probabilities = F.softmax(output, dim=1).squeeze().cpu().numpy()
        predicted = int(output.argmax(1).item())

    # Lấy ground-truth
    original_name = cropped_img.stem.rsplit("_", 1)[0]
    box_index = int(cropped_img.stem.rsplit("_", 1)[1])
    gt_label_path = labels_folder / f"{original_name}.txt"
    with open(gt_label_path, 'r') as f:
        gt_line = f.readlines()[box_index]
        gt_class = int(gt_line.strip().split()[0])

    # Lưu kết quả cho Cell 2
    results.append({
        "image": cropped_img.name,
        "gt": gt_class,
        "pred": predicted,
        "confidence": float(probabilities[predicted]),
        "probabilities": probabilities.tolist()
    })

    # ====== Visualization từng ảnh ======
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))

    # Hiển thị ảnh
    ax1.imshow(img)
    ax1.axis("off")
    ax1.set_title(f"{'✅' if predicted == gt_class else '❌'} GT: {gt_class} | Pred: {predicted}",
                  color='green' if predicted == gt_class else 'red')

    # Biểu đồ xác suất
    ax2.bar(range(len(probabilities)), probabilities)
    ax2.set_xticks(range(len(probabilities)))
    ax2.set_title("Confidence per class")
    ax2.set_xlabel("Class ID")
    ax2.set_ylabel("Confidence")

    plt.tight_layout()
    plt.show()

# Lưu kết quả dưới dạng dataframe để visualize sau
df_results = pd.DataFrame(results)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# Tính tổng accuracy
acc = accuracy_score(df_results["gt"], df_results["pred"])
print(f"🔍 Final Accuracy: {acc:.2%}")

# Biểu đồ confusion matrix
cm = confusion_matrix(df_results["gt"], df_results["pred"])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

# Biểu đồ đúng/sai theo confidence
df_results["correct"] = df_results["gt"] == df_results["pred"]

plt.figure(figsize=(10, 5))
plt.hist([df_results[df_results["correct"]==True]["confidence"],
          df_results[df_results["correct"]==False]["confidence"]],
         label=["Correct", "Incorrect"], bins=10, alpha=0.7)
plt.xlabel("Prediction Confidence")
plt.ylabel("Number of Samples")
plt.title("Confidence Distribution")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
model.fc.out_features